# Boosting para el dataset EuroSAT

# 1. Acceso a drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2. Importando módulos

In [ ]:
import pandas as pd
import os
from ____ import LabelEncoder
from ____ import MinMaxScaler
from ____ import DecisionTreeClassifier
from sklearn import metrics
import numpy as np
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

# 3. Lectura de los datos

In [ ]:
path = ______
train_path = os.path.join(path,'EUROSAT_TRAIN_FEAT.csv')

In [ ]:
train_df = pd._____(train_path)
clases = train_df['label'].unique()
train_df.head()

**Enlistando las categorías de las imágenes**

In [ ]:
clases = train_df['label'].unique().tolist()
n_clases = len(clases)
print(clases)

**Visualizando distribución de clases**

In [ ]:
fig, ax =plt.subplots(figsize=(16,5))
sns.countplot('label', data=____)
plt.title('Distribucion de clases de Eurosat')
plt.xlabel('Clase', fontsize=14)
plt.ylabel('# de imágenes', fontsize=14)
plt.show()

# 3. Codificación de las etiquetas y escalamiento

**Codificación de etiquetas de la columna label**

In [ ]:
le = _____
train_df['label'] = ___.____(train_df.label.values)

**Escalamiento de las características de 0 a 1**

In [ ]:
scaler = _____(____=(0, 1))
train_df.loc[:, train_df.columns != 'label'] = scaler._____(train_df.loc[:, train_df.columns != 'label'])

# 4. Análisis de componentes principales

**Análisis PCA con el 95% de varianza acumulada**

In [ ]:
pca = ___(0.95)
pc = ___._____(train_df.iloc[:,:-1])
df_pca = pd.DataFrame(data = pc,
                           columns=range(pc.shape[1]))
df_pca = pd.concat([df_pca, train_df.iloc[:,-1]], axis = 1)

In [ ]:
print('Número de componentes principales: %s'%len(pca.explained_variance_ratio_))
print('Varianza acumulada con %s componentes: %s'%(len(pca.explained_variance_ratio_),np.sum(pca.explained_variance_ratio_)))

In [ ]:
df_pca.head()

# 5. Validación para los modelos

**Dividiendo las características (vars indeps) de la etiqueta (var dep)**

In [ ]:
X = np.array(____) 
y = np.array(____) 

**Hold- Out validation con 20% de datos para prueba**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(____, ____, test_size = ____, random_state = 0, shuffle=____)

print(f'Dimension de los datos de entrenamiento {X_train.shape}')
print(f'Dimension de los datos de prueba {X_test.shape}')

# 6. Árbol de decisión (baseline)

**Creación de nuestro modelo sencillo con 5 niveles de decisión, de aquí partimos**

In [ ]:
cart =  DecisionTreeClassifier(max_depth=___)

**Entrenamiento con los datos de entrenamiento**

In [ ]:
cart = ____.___(____, ____)

**Score de entrenmiento**

In [ ]:
___.score(____,____)

**Obteniendo las predicciones**

In [ ]:
y_pred = ____.___(____)

**Score de los datos de prueba i.e. los predichos**

In [ ]:
score = metrics.accuracy_score(____, ____)
print("Test Acc: %s"%score)

**Matri de confusión**

In [ ]:
def plot_confusion_matrix(cm, classes, tit, normalize=False):
    if normalize:
        cm = cm.astype('float')/cm.sum(axis=1)
        title, fmt = 'Matriz de confusión normalizada', '.2f'
    else:
        title, fmt = tit, 'd'
    plt.figure(figsize=(10,8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title)#, fontsize=12)
    plt.colorbar(pad=0.05)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=40)
    plt.yticks(tick_marks, classes)
    thresh = cm.max()/2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),horizontalalignment="center", 
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('Clase Verdadera')
    plt.xlabel('Clase Predicha')
    plt.savefig(title+'.png')
    plt.grid(False)
    plt.show()

**¿Qué observamos?**

In [ ]:
cnf_matrix = confusion_matrix(y_test, y_pred, labels=range(n_clases))
tit = 'Matriz de confusión CART'
plot_confusion_matrix(cnf_matrix,clases, tit, normalize=False)

# ¿Cuántos weak learners son necesarios?

In [ ]:
Xclassifiers = []
Yscore = []

# Arbol de decisión sencillo
cart =  DecisionTreeClassifier(max_depth=5)
for i in range(500):
  if i%100 == 0:
    # En cada iteración, se creará una adaboost con i+1 árboles sencillos
    boost = AdaBoostClassifier(base_estimator=_____, n_estimators=____)
    # Entrenando el modelo
    boost = boost.fit(____, ____)
    # Prediciendo los de prueba
    preds = boost.predict(___)
    # Score de prueba
    score = metrics.accuracy_score(___, ____)
    Yscore.append(score)
    Xclassifiers.append(i+1)
    print("iter: %s, score: %s"%(i+1,np.mean(score)))


In [ ]:
plt.figure()
plt.plot(Xclassifiers, Yscore, lw=2)
plt.xlabel('Clasificadores')
plt.ylabel('Score Promedio')
plt.title('Score vs Classificadores: AdaBoost + CART')
print("Score máximo alcanzado: ",np.max(Yscore)) 
plt.grid()
plt.show() 

# 7. Boosting con CART

**Creando nuestro clasificador de Boosting**

In [ ]:
cart = _____(max_depth=____)
boost = ______(base_estimator=____, n_estimators=___)

**Entrenando al clasificador**

In [ ]:
boost = ____.____(____, ____)

**Score de entrenamiento**

In [ ]:
___.___(___,___)

**Predicciones**

In [ ]:
y_pred = boost.____(____)

**Score de prueba**

In [ ]:
score = metrics.accuracy_score(y_test, ____)
score

**Matriz de confusión**

In [ ]:
cnf_matrix = confusion_matrix(____, ____, labels=range(n_clases))
tit = 'Matriz de confusión AdaBoost'
plot_confusion_matrix(cnf_matrix,clases, tit, normalize=False)